# 프롬프트 엔지니어링 실습

이 실습에서는 다양한 프롬프트 엔지니어링 기법을 사용하여 과학과 수학 문제를 해결해보겠습니다.
각 기법의 특징을 이해하고, 문제 유형에 따른 최적의 접근 방법을 학습하는 것이 목표입니다.

## 다룰 기법들
1. **Zero-shot**: 예시 없이 바로 문제 해결
2. **Few-shot**: 몇 가지 예시로 패턴 학습
3. **Chain-of-Thought (CoT)**: 단계별 추론 과정 명시
4. **Self-Consistency**: 여러 번 풀어서 일관된 답 선택
6. **Tree-of-Thought (ToT)**: 여러 사고 경로 탐색

In [3]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [4]:
# OpenAI 사용하기
from openai import OpenAI
import time
from collections import Counter

# OpenAI 클라이언트 초기화
# client = OpenAI(api_key=api_key)
client = OpenAI()
model = "gpt-4o-mini"

# 기본 함수들
def get_response(messages, temperature=0, max_tokens=5000):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def print_result(technique, problem, response):
    print(f"\n{'='*60}")
    print(f"기법: {technique}")
    print(f"{'='*60}")
    print(f"문제: {problem}")
    print(f"\n답변:\n{response}")
    print(f"{'='*60}")

print("환경 설정이 완료되었습니다!")

환경 설정이 완료되었습니다!


In [10]:
messages = [
    {"role": "user", "content": "발 없는 말이?"}
]

response = get_response(messages, temperature=.4)

## 1. Zero-shot 프롬프트

어떤 예시도 제공하지 않고 모델에게 직접 문제를 해결하도록 지시하는 가장 기본적인 방법입니다.
모델의 사전 지식과 일반화 능력에만 의존합니다.

In [5]:
# Zero-shot 예제 1: 중국 당나라 한시 스타일 크리스마스 캐롤
problem_1 = """중국 당나라 때 한시 스타일로 크리스마스 캐롤을 만드세요. 한자 아래에 음독을 추가하세요.
"""

messages = [
    {"role": "user", "content": problem_1}
]

response = get_response(messages)
print_result("Zero-shot", problem_1, response)


기법: Zero-shot
문제: 중국 당나라 때 한시 스타일로 크리스마스 캐롤을 만드세요. 한자 아래에 음독을 추가하세요.


답변:
당나라 스타일의 한시로 크리스마스 캐롤을 만들어 보겠습니다. 아래는 한시와 그 음독입니다.

---

雪花飘舞夜明辉  
(설화 표무 야명 휘)  
星辰闪烁照人归  
(성진 섬작 조인 귀)  
圣诞佳音传四方  
(성탄 가음 전 사방)  
欢声笑语共此时  
(환성 소어 공 차시)  

---

이 시는 크리스마스의 기쁨과 축제를 표현하고 있습니다. 눈꽃이 날리는 밤, 별들이 빛나며 사람들을 맞이하고, 성탄의 좋은 소식이 사방에 퍼지며, 즐거운 목소리와 웃음이 함께하는 순간을 담고 있습니다.


In [7]:
# Zero-shot 예제 2: 2차 방정식 풀이
problem_2 = "다음 2차 방정식을 풀어주세요: 2x² - 7x + 3 = 0"

messages = [
    {"role": "user", "content": problem_2}
]

response = get_response(messages)
print_result("Zero-shot", problem_2, response)


기법: Zero-shot
문제: 다음 2차 방정식을 풀어주세요: 2x² - 7x + 3 = 0

답변:
주어진 2차 방정식 \( 2x^2 - 7x + 3 = 0 \)을 풀기 위해, 2차 방정식의 근의 공식을 사용할 수 있습니다. 근의 공식은 다음과 같습니다:

\[
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\]

여기서 \( a = 2 \), \( b = -7 \), \( c = 3 \)입니다. 이 값을 대입해 보겠습니다.

1. 판별식 \( D = b^2 - 4ac \) 계산:
   \[
   D = (-7)^2 - 4 \cdot 2 \cdot 3 = 49 - 24 = 25
   \]

2. 근의 공식에 대입:
   \[
   x = \frac{-(-7) \pm \sqrt{25}}{2 \cdot 2} = \frac{7 \pm 5}{4}
   \]

3. 두 개의 해를 구합니다:
   - 첫 번째 해:
   \[
   x_1 = \frac{7 + 5}{4} = \frac{12}{4} = 3
   \]
   - 두 번째 해:
   \[
   x_2 = \frac{7 - 5}{4} = \frac{2}{4} = \frac{1}{2}
   \]

따라서, 주어진 2차 방정식 \( 2x^2 - 7x + 3 = 0 \)의 해는 \( x = 3 \)과 \( x = \frac{1}{2} \)입니다.


In [8]:
# Zero-shot 예제 3: Zero-shot 분류
problem_3 = """다음 문장들을 날씨로 표현해주세요:

1. "오늘 시험을 망쳤어요"
2. "새로운 직장에서 첫 출근을 해요"
3. "오랜만에 친구들과 만났어요"
4. "연인과 헤어졌어요"
5. "로또에 당첨됐어요"
"""

messages = [
    {"role": "user", "content": problem_3}
]

response = get_response(messages)
print_result("Zero-shot", problem_3, response)


기법: Zero-shot
문제: 다음 문장들을 날씨로 표현해주세요:

1. "오늘 시험을 망쳤어요"
2. "새로운 직장에서 첫 출근을 해요"
3. "오랜만에 친구들과 만났어요"
4. "연인과 헤어졌어요"
5. "로또에 당첨됐어요"


답변:
1. "오늘 시험을 망쳤어요" - 흐리고 비가 내리는 날, 기분이 우울한 날씨.
2. "새로운 직장에서 첫 출근을 해요" - 맑고 햇살이 비치는 상쾌한 아침, 새로운 시작을 알리는 날씨.
3. "오랜만에 친구들과 만났어요" - 따뜻한 봄바람이 부는 날, 화창하고 기분 좋은 날씨.
4. "연인과 헤어졌어요" - 차가운 바람이 불고, 구름이 가득한 우울한 날씨.
5. "로또에 당첨됐어요" - 맑고 푸른 하늘, 햇살이 반짝이는 기쁜 날씨.


모델이 학습하지 않은 문제에 대해서 물어본다면 어떻게 답변할까요?

In [9]:
# Zero-shot 예제 4: 임의 연산자 방정식 풀이
problem_4 = "다음 방정식을 풀어주세요: 2 @ 7 = ?"

messages = [
    {"role": "user", "content": problem_4}
]

response = get_response(messages)
print_result("Zero-shot", problem_3, response)


기법: Zero-shot
문제: 다음 문장들을 날씨로 표현해주세요:

1. "오늘 시험을 망쳤어요"
2. "새로운 직장에서 첫 출근을 해요"
3. "오랜만에 친구들과 만났어요"
4. "연인과 헤어졌어요"
5. "로또에 당첨됐어요"


답변:
주어진 방정식 "2 @ 7 = ?"에서 '@' 기호의 의미가 명확하지 않습니다. '@'가 어떤 연산을 나타내는지에 대한 정보가 필요합니다. 예를 들어, '@'가 덧셈, 뺄셈, 곱셈, 나눗셈 또는 다른 특정한 연산을 의미할 수 있습니다. '@'의 의미를 알려주시면 방정식을 풀어드리겠습니다.


세상에 없는 연산자에 대해서는 학습되지 않았기 때문에 기호의 의미를 물어봅니다.

## 2. Few-shot 프롬프트

몇 가지 예시를 제공하여 모델이 패턴을 학습하도록 하는 방법입니다.
Zero-shot보다 일관성 있고 정확한 결과를 얻을 수 있습니다.

In [13]:
# Few-shot 예제 1: 문장을 날씨로 표현하기 (분류 태깅)
few_shot_prompt_1 = """다음 예시를 참고하여 문장들을 날씨로 표현해주세요:

예시:
입력: "승진 소식을 들었어요."
출력: 맑음

입력: "시험에 떨어졌어요."  
출력: 비

입력: "새로운 도전을 앞두고 있어요."
출력: 안개

이제 다음 문장들을 같은 방식으로 표현해주세요:
1. "몸이 많이 아파요."
2. "충격적인 소식을 들었어요"
3. "시험에 합격했어요."
4. "친구와 대판 싸웠어요."
"""

messages = [
    {"role": "user", "content": few_shot_prompt_1}
]

response = get_response(messages)
print_result("Few-shot", "날씨 분류", response)


기법: Few-shot
문제: 날씨 분류

답변:
1. 흐림
2. 폭풍
3. 맑음
4. 비


In [12]:
# Few-shot 예제 2: 유기화학 명명법
few_shot_prompt_2 = """
다음은 유기화합물의 IUPAC 명명법 예시입니다:

예시 1:
구조: CH₃-CH₂-CH₂-CH₃
분석: 탄소 4개의 직쇄 알케인, 이중결합 없음
명명: 부테인 (butane)

예시 2:
구조: CH₃-CH(CH₃)-CH₂-CH₃
분석: 탄소 4개의 주쇄, 2번 탄소에 메틸기 치환
명명: 2-메틸부테인 (2-methylbutane)

예시 3:
구조: CH₃-CH₂-CH(OH)-CH₃
분석: 탄소 4개의 주쇄, 2번 탄소에 히드록시기(-OH)
명명: 2-부탄올 (2-butanol)

이제 다음 화합물의 IUPAC 이름을 같은 형식으로 명명해주세요:
구조: CH₃-CH₂-CH(CH₃)-CH₂-CH(CH₃)-CH₃
"""

messages = [
    {"role": "user", "content": few_shot_prompt_2}
]

response = get_response(messages)
print_result("Few-shot", "유기화학 명명법", response)


기법: Few-shot
문제: 유기화학 명명법

답변:
구조: CH₃-CH₂-CH(CH₃)-CH₂-CH(CH₃)-CH₃  
분석: 탄소 6개의 주쇄, 3번과 5번 탄소에 각각 메틸기 치환  
명명: 3,5-디메틸헥세인 (3,5-dimethylhexane)


In [14]:
# Few-shot 예제 3: 임의 연산자 방정식 풀이
few_shot_prompt_3 = """
다음은 수학 방정식을 푸는 예시입니다:

예시 1:
문제: 3 @ 8 = ?
답: 25

예시 2:
문제: 4 @ 9 = ?
답: 37

예시 3:
문제: 6 @ 2 = ?
답: 13

이제 다음 문제를 같은 형식으로 풀어주세요:
구조: 234 @ 75 = ?
"""

messages = [
    {"role": "user", "content": few_shot_prompt_3}
]

response = get_response(messages)
print_result("Few-shot", "임의 연산자 방정식 문제", response)


기법: Few-shot
문제: 임의 연산자 방정식 문제

답변:
주어진 예시들을 분석해보면, '@' 기호가 특정한 연산을 나타내는 것 같습니다. 각 예시에서 '@' 연산의 결과를 도출하는 규칙을 찾아보겠습니다.

1. 3 @ 8 = 25
2. 4 @ 9 = 37
3. 6 @ 2 = 13

이 예시들을 통해 '@' 연산의 규칙을 유추해보면, 다음과 같은 패턴이 보입니다:

- 3 @ 8 = 3 * 8 + 1 = 24 + 1 = 25
- 4 @ 9 = 4 * 9 + 1 = 36 + 1 = 37
- 6 @ 2 = 6 * 2 + 1 = 12 + 1 = 13

따라서 '@' 연산은 두 수를 곱한 후 1을 더하는 것으로 보입니다.

이제 주어진 문제를 풀어보겠습니다:

문제: 234 @ 75 = ?
풀이: 234 * 75 + 1 = 17550 + 1 = 17551

답: 17551


이렇게 세상에 없는 연산자라도 few-shot으로 예시를 제공하면 예시 속에서 규칙을 찾아 답을 논리적으로 도출하는 것을 확인할 수 있습니다.

## 3. Chain-of-Thought (CoT) 프롬프트

단계별 추론 과정을 명시적으로 요구하여 복잡한 문제의 정확도를 크게 향상시키는 방법입니다.
"단계별로 생각해보세요" 같은 지시를 통해 모델이 논리적 사고 과정을 거치도록 유도합니다.

In [15]:
# CoT 예제 1: 조합과 확률 문제
cot_problem_1 = """
다음 문제를 단계별로 풀어주세요:

문제: 5명의 학생 중에서 3명을 선발하는 방법의 수는 몇 가지인가요?
그리고 이 5명 중에서 회장 1명, 부회장 1명, 총무 1명을 뽑는 방법의 수는 몇 가지인가요?

각 단계를 명확히 설명하고 공식을 사용한 이유도 함께 설명해주세요.
"""

messages = [
    {"role": "user", "content": cot_problem_1}
]

response = get_response(messages)
print_result("Chain-of-Thought", "조합과 확률 문제", response)


기법: Chain-of-Thought
문제: 조합과 확률 문제

답변:
이 문제를 단계별로 풀어보겠습니다.

### 1단계: 5명 중에서 3명을 선발하는 방법의 수

**문제 이해**: 5명의 학생 중에서 3명을 선택하는 경우의 수를 구해야 합니다. 이 경우, 선택된 학생의 순서는 중요하지 않으므로 조합(combination) 공식을 사용합니다.

**조합 공식**: 
\[
C(n, r) = \frac{n!}{r!(n-r)!}
\]
여기서 \( n \)은 전체 학생 수, \( r \)은 선택할 학생 수, \( ! \)는 팩토리얼을 의미합니다.

**계산**:
- \( n = 5 \) (전체 학생 수)
- \( r = 3 \) (선택할 학생 수)

공식에 대입하면:
\[
C(5, 3) = \frac{5!}{3!(5-3)!} = \frac{5!}{3! \cdot 2!}
\]

이제 각 팩토리얼을 계산합니다:
- \( 5! = 5 \times 4 \times 3 \times 2 \times 1 = 120 \)
- \( 3! = 3 \times 2 \times 1 = 6 \)
- \( 2! = 2 \times 1 = 2 \)

따라서,
\[
C(5, 3) = \frac{120}{6 \cdot 2} = \frac{120}{12} = 10
\]

**결과**: 5명의 학생 중에서 3명을 선발하는 방법의 수는 **10가지**입니다.

---

### 2단계: 5명 중에서 회장 1명, 부회장 1명, 총무 1명을 뽑는 방법의 수

**문제 이해**: 5명의 학생 중에서 회장, 부회장, 총무를 각각 1명씩 선발하는 경우의 수를 구해야 합니다. 이 경우, 각 직책의 순서가 중요하므로 순열(permutation) 공식을 사용합니다.

**순열 공식**:
\[
P(n, r) = \frac{n!}{(n-r)!}
\]
여기서 \( n \)은 전체 학생 수, \( r \)은 선택할 직책 수입니다.

**계산**:
- \( n = 5 \) (전체 학생 수)
- \( r = 3 \) (선택

In [16]:
# CoT 예제 2: 기하 문제 (정답=5√3)
cot_problem_2 = """다음 문제를 단계별로 풀어주세요:

문제: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하세요.

해결 과정:
1. 문제 상황 이해
2. 좌표계 설정 또는 기하학적 관계 파악
3. 정삼각형의 성질 활용
4. 삼각함수 또는 피타고라스 정리 적용
5. 최종 답 계산 및 검증

각 단계를 차근 차근 분석하여 푸세요. 자세한 계산 과정을 보여주세요.
"""

messages = [
    {"role": "user", "content": cot_problem_2}
]

response = get_response(messages)
print_result("Chain-of-Thought", "화학 평형 문제", response)


기법: Chain-of-Thought
문제: 화학 평형 문제

답변:
문제를 단계별로 풀어보겠습니다.

### 1. 문제 상황 이해
주어진 문제는 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하는 것입니다. 내접하는 정삼각형은 원의 중심에서 각 꼭짓점까지의 거리가 모두 반지름과 같고, 각 변은 원의 둘레에 접합니다.

### 2. 좌표계 설정 또는 기하학적 관계 파악
정삼각형의 중심을 원의 중심과 일치시키고, 원의 반지름을 기준으로 정삼각형의 꼭짓점을 설정합니다. 정삼각형의 각 꼭짓점은 원의 둘레에 위치하게 됩니다.

정삼각형의 각 내각은 60도입니다. 따라서, 정삼각형의 각 꼭짓점에서 원의 중심까지의 선분은 5cm의 길이를 가집니다.

### 3. 정삼각형의 성질 활용
정삼각형의 성질에 따르면, 정삼각형의 한 변의 길이를 \( a \)라고 할 때, 정삼각형의 높이는 다음과 같이 구할 수 있습니다:

\[
h = \frac{\sqrt{3}}{2} a
\]

또한, 정삼각형의 높이는 원의 반지름과 관련이 있습니다. 정삼각형의 높이는 원의 반지름과 같으므로:

\[
h = R = 5 \text{ cm}
\]

### 4. 삼각함수 또는 피타고라스 정리 적용
정삼각형의 높이를 \( h \)로 두고, 정삼각형의 한 변의 길이를 \( a \)로 두면, 다음과 같은 관계가 성립합니다:

\[
h = \frac{\sqrt{3}}{2} a
\]

여기서 \( h = 5 \) cm이므로, 이를 대입하여 \( a \)를 구합니다:

\[
5 = \frac{\sqrt{3}}{2} a
\]

양변에 2를 곱하면:

\[
10 = \sqrt{3} a
\]

이제 양변을 \( \sqrt{3} \)로 나누면:

\[
a = \frac{10}{\sqrt{3}}
\]

### 5. 최종 답 계산 및 검증
위의 식에서 \( a \)를 계산합니다. \( \sqrt{3} \)의 근사값을 1.732로 사용하여 계산하면:

\[
a \approx \frac{10}{1.732} \a

In [17]:
# CoT 예제 3: 공간 추론 문제
cot_problem_3 = """
다음 문제를 단계별로 풀어주세요:

문제: 침실에 있는 컵 안에 공을 넣었습니다.
컵을 거실로 옮겼습니다.
컵을 뒤집어 놓았습니다.
다시 컵을 잡고 부엌으로 갔습니다.
공은 지금 어디에 있을까요?


각 단계를 명확히 설명하고 그렇게 생각한 이유도 함께 설명해주세요.
"""

messages = [
    {"role": "user", "content": cot_problem_3}
]

response = get_response(messages)
print_result("Chain-of-Thought", "공간 추론 문제", response)


기법: Chain-of-Thought
문제: 공간 추론 문제

답변:
이 문제를 단계별로 풀어보겠습니다.

1. **첫 번째 단계: 침실에 있는 컵 안에 공을 넣었습니다.**
   - 이 단계에서 컵 안에 공이 들어있습니다. 따라서 현재 상태는 "컵 안에 공이 있다"입니다.

2. **두 번째 단계: 컵을 거실로 옮겼습니다.**
   - 컵이 거실로 옮겨졌지만, 컵 안에 있는 공은 여전히 컵 안에 있습니다. 따라서 현재 상태는 "컵 안에 공이 있고, 컵은 거실에 있다"입니다.

3. **세 번째 단계: 컵을 뒤집어 놓았습니다.**
   - 컵을 뒤집으면 컵 안에 있던 공이 컵 밖으로 떨어지게 됩니다. 따라서 현재 상태는 "컵은 거실에 있고, 컵 안에는 공이 없다"입니다. 공은 거실 바닥에 떨어져 있을 것입니다.

4. **네 번째 단계: 다시 컵을 잡고 부엌으로 갔습니다.**
   - 컵을 부엌으로 옮겼지만, 공은 여전히 거실 바닥에 떨어져 있습니다. 컵이 부엌으로 이동했더라도 공의 위치에는 영향을 미치지 않습니다.

결론적으로, 공은 지금 **거실 바닥에** 있습니다. 컵이 부엌으로 이동했지만, 공은 컵이 뒤집혔을 때 떨어졌기 때문에 컵의 위치와는 무관하게 거실에 남아 있습니다.


In [18]:
# CoT 예제 4: 복잡한 추론 문제
cot_problem_4 = """
다음 문제를 단계별로 풀어주세요:

문제: Alexander, Jackson, 그리고 Benjamin은 함께 차고에 들어갔습니다.
사과는 처음에 녹색 서랍에 있었습니다.
Jackson은 차고에서 나갔고, 그 이후에 Alexander는 사과를 파란 상자로 옮겼습니다.
나중에 Alexander, Jackson, Benjamin은 다시 차고에 들어왔습니다.
Alexander는 Jackson이 Benjamin이 사과를 어디에서 찾을 것이라고 생각한다고 생각하나요?

각 단계를 명확히 설명하고 그렇게 생각한 이유도 함께 설명해주세요.
"""

messages = [
    {"role": "user", "content": cot_problem_4}
]

response = get_response(messages)
print_result("Chain-of-Thought", "복잡한 추론 문제", response)


기법: Chain-of-Thought
문제: 복잡한 추론 문제

답변:
이 문제를 단계별로 풀어보겠습니다.

### 1단계: 상황 이해하기
- **사람들**: Alexander, Jackson, Benjamin
- **사과의 위치**: 처음에 녹색 서랍에 있음
- **사건**:
  1. Jackson이 차고에서 나감
  2. Alexander가 사과를 파란 상자로 옮김
  3. 나중에 Alexander, Jackson, Benjamin이 다시 차고에 들어옴

### 2단계: 사건의 흐름 정리하기
- 처음에 사과는 녹색 서랍에 있었습니다.
- Jackson이 차고를 나간 후, Alexander는 사과를 파란 상자로 옮겼습니다. 이 시점에서 사과의 위치는 파란 상자입니다.
- 그 후, Alexander, Jackson, Benjamin이 다시 차고에 들어왔습니다. 이때 Jackson은 사과의 위치를 알지 못합니다.

### 3단계: Jackson의 지식 상태 분석하기
- Jackson은 차고를 나갔기 때문에 사과가 파란 상자로 옮겨진 사실을 모릅니다. 그는 여전히 사과가 녹색 서랍에 있다고 생각할 수 있습니다.
- Alexander는 Jackson이 Benjamin이 사과를 찾을 때, 사과가 여전히 녹색 서랍에 있다고 생각할 것이라고 추측할 수 있습니다.

### 4단계: Alexander의 생각 정리하기
- Alexander는 Jackson이 사과의 현재 위치를 모르기 때문에, Jackson이 Benjamin에게 "사과는 녹색 서랍에 있을 것"이라고 말할 것이라고 생각할 수 있습니다.
- 따라서 Alexander는 Jackson이 Benjamin이 사과를 찾을 때, 녹색 서랍을 찾을 것이라고 예상할 것입니다.

### 결론
Alexander는 Jackson이 Benjamin이 사과를 찾을 때, 사과가 녹색 서랍에 있다고 생각할 것이라고 추측합니다. 이는 Jackson이 사과의 위치를 모르는 상태에서 자연스럽게 발생하는 추론입니다.


## 4. Self-Consistency

같은 문제를 여러 번 다른 방식으로 풀어보고, 가장 일관된 답을 선택하는 방법입니다.
Chain-of-Thought의 안정성을 높이는 기법입니다.

In [19]:
# Self-Consistency 예제 1: 기하학 문제
geometry_problem = """
다음 문제를 단계별로 풀어주세요:

문제: 반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하세요.

단계별로 해결 과정을 보여주고, 최종 답을 명확히 제시해주세요.
"""

# 여러 번 풀어보기
responses = []
final_answers = []

print("같은 문제를 5번 다른 방식으로 풀어보겠습니다...\n")

for i in range(5):
    messages = [
        {"role": "user", "content": geometry_problem}
    ]

    response = get_response(messages, temperature=0.3)  # 약간의 변동성 추가
    responses.append(response)

    print(f"풀이 {i+1}:")
    print(response[:200] + "...")
    print()

    # 답 추출 시도 (간단한 패턴 매칭)
    if "cm" in response.lower():
        import re
        numbers = re.findall(r'\d+\.?\d*', response)
        if numbers:
            final_answers.append(numbers[-1])  # 마지막 숫자를 답으로 가정

    time.sleep(1)  # API 호출 간격

print(f"\n추출된 답들: {final_answers}")
if final_answers:
    answer_counts = Counter(final_answers)
    most_common = answer_counts.most_common(1)[0]
    print(f"가장 일관된 답: {most_common[0]} ({most_common[1]}번 출현)")

같은 문제를 5번 다른 방식으로 풀어보겠습니다...

풀이 1:
반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하는 문제를 단계별로 풀어보겠습니다.

### 단계 1: 정사각형의 성질 이해하기
정사각형이 원에 내접한다는 것은 정사각형의 모든 꼭짓점이 원 위에 위치한다는 의미입니다. 이 경우, 정사각형의 대각선은 원의 지름과 같습니다.

### 단계 2: 원의 지름 구하기
원에 내접하는 정사각형의 대각선은 ...

풀이 2:
반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하는 문제를 단계별로 해결해 보겠습니다.

### 단계 1: 원과 정사각형의 관계 이해하기
내접하는 정사각형은 원 안에 완전히 들어가며, 정사각형의 모든 꼭짓점이 원의 둘레에 위치합니다. 이때, 정사각형의 대각선은 원의 지름과 같습니다.

### 단계 2: 원의 지름 구하기
반지름이 5cm인 원의...

풀이 3:
반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하는 문제를 단계별로 해결해 보겠습니다.

### 단계 1: 원과 정사각형의 관계 이해하기
내접하는 정사각형은 원의 중심을 기준으로 대칭적으로 위치합니다. 정사각형의 각 꼭짓점은 원의 둘레에 위치하게 됩니다. 이때, 정사각형의 대각선은 원의 지름과 같습니다.

### 단계 2: 원의 지름 구하기
...

풀이 4:
반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하는 문제를 단계별로 해결해 보겠습니다.

### 단계 1: 문제 이해하기
- 원의 반지름이 5cm입니다.
- 이 원에 내접하는 정사각형을 생각합니다. 내접하는 정사각형의 각 꼭짓점은 원의 둘레에 위치하게 됩니다.

### 단계 2: 정사각형의 대각선과 원의 반지름 관계 이해하기
- 정사각형의 대...

풀이 5:
반지름이 5cm인 원에 내접하는 정사각형의 한 변의 길이를 구하는 문제를 단계별로 해결해 보겠습니다.

### 단계 1: 정사각형의 성질 이해하기
정사각형이 원에 내접한다는 것은 정사각형의 모든 꼭짓점이 원 위에 위

In [20]:
# Self-Consistency 예제 2: 열역학 문제
thermo_problem = """
다음 열역학 문제를 풀어주세요:

문제: 이상기체 1mol이 300K, 2atm에서 600K, 1atm로 변화했습니다.
이 과정에서 엔트로피 변화(ΔS)를 구하세요. (R = 8.314 J/mol·K)

단계별 풀이와 함께 최종 답을 J/mol·K 단위로 제시해주세요.
"""

# 3번 풀어보기
thermo_responses = []
thermo_answers = []

print("열역학 문제를 3번 풀어보겠습니다...\n")

for i in range(3):
    messages = [
        {"role": "user", "content": thermo_problem}
    ]

    response = get_response(messages, temperature=0.2)
    thermo_responses.append(response)

    print(f"풀이 {i+1}:")
    print(response)
    print(f"{'='*50}\n")

    time.sleep(1)

print("Self-Consistency 분석 완료!")

열역학 문제를 3번 풀어보겠습니다...

풀이 1:
이상기체의 엔트로피 변화(ΔS)를 구하기 위해서는 두 가지 과정으로 나눌 수 있습니다: 온도 변화와 압력 변화. 이상기체의 엔트로피 변화는 다음과 같은 식으로 계산할 수 있습니다.

1. **온도 변화에 따른 엔트로피 변화 (ΔS_T)**:
   \[
   \Delta S_T = nR \ln\left(\frac{T_2}{T_1}\right)
   \]
   여기서 \( n \)은 몰 수, \( R \)은 기체 상수, \( T_1 \)과 \( T_2 \)는 각각 초기와 최종 온도입니다.

2. **압력 변화에 따른 엔트로피 변화 (ΔS_P)**:
   \[
   \Delta S_P = -nR \ln\left(\frac{P_2}{P_1}\right)
   \]
   여기서 \( P_1 \)과 \( P_2 \)는 각각 초기와 최종 압력입니다.

이제 주어진 값을 대입하여 계산해 보겠습니다.

### 1단계: 온도 변화에 따른 엔트로피 변화 계산

- \( n = 1 \, \text{mol} \)
- \( R = 8.314 \, \text{J/(mol·K)} \)
- \( T_1 = 300 \, \text{K} \)
- \( T_2 = 600 \, \text{K} \)

\[
\Delta S_T = 1 \times 8.314 \ln\left(\frac{600}{300}\right)
\]
\[
= 8.314 \ln(2)
\]
\[
\approx 8.314 \times 0.693 \approx 5.764 \, \text{J/(mol·K)}
\]

### 2단계: 압력 변화에 따른 엔트로피 변화 계산

- \( P_1 = 2 \, \text{atm} \)
- \( P_2 = 1 \, \text{atm} \)

\[
\Delta S_P = -1 \times 8.314 \ln\left(\frac{1}{2}\right)
\]
\[
= -8.314 \ln(0.5)
\]
\[
= -8.314 \times (-0.693) \appr

## 6. Tree-of-Thought (ToT)

여러 사고 경로를 동시에 탐색하고 평가하여 최적의 해결책을 찾는 방법입니다.
복잡한 문제에서 다양한 접근법을 시도해볼 수 있습니다.

In [21]:
# Tree-of-Thought 예제 1: 최적화 문제
tot_problem_1 = """
다음 최적화 문제를 Tree-of-Thought 방식으로 해결해주세요:

문제: 직사각형 모양의 정원(가로 x, 세로 y)을 만들려고 합니다.
둘레가 40m일 때 면적이 최대가 되는 x, y 값을 구하세요.

다음과 같이 여러 경로를 탐색해주세요:

경로 1: 미분을 이용한 접근
경로 2: 기하학적 성질을 이용한 접근
경로 3: 산술-기하 평균을 이용한 접근

각 경로를 평가하고 가장 효율적인 방법을 선택해주세요.
"""

messages = [
    {"role": "user", "content": tot_problem_1}
]

response = get_response(messages)
print_result("Tree-of-Thought", "최적화 문제", response)


기법: Tree-of-Thought
문제: 최적화 문제

답변:
최적화 문제를 Tree-of-Thought 방식으로 해결해 보겠습니다. 주어진 문제는 직사각형 정원의 둘레가 40m일 때 면적을 최대화하는 x와 y 값을 찾는 것입니다. 각 경로를 탐색해 보겠습니다.

### 경로 1: 미분을 이용한 접근

1. **문제 설정**: 
   - 둘레 \( P = 2x + 2y = 40 \)
   - 면적 \( A = xy \)

2. **둘레 식을 면적 식에 대입**:
   - \( x + y = 20 \) (둘레 식을 변형)
   - \( y = 20 - x \)

3. **면적을 x에 대한 함수로 표현**:
   - \( A(x) = x(20 - x) = 20x - x^2 \)

4. **미분하여 극대값 찾기**:
   - \( A'(x) = 20 - 2x \)
   - \( A'(x) = 0 \)일 때 \( 20 - 2x = 0 \) → \( x = 10 \)

5. **y 값 계산**:
   - \( y = 20 - x = 10 \)

6. **결론**:
   - 최대 면적을 가지는 직사각형의 크기는 \( x = 10m, y = 10m \)이며, 면적은 \( 100m^2 \)입니다.

### 경로 2: 기하학적 성질을 이용한 접근

1. **문제 설정**:
   - 둘레가 일정할 때 면적이 최대가 되는 도형은 정사각형입니다.

2. **정사각형의 성질**:
   - 둘레 \( P = 4s \) (s는 한 변의 길이)
   - \( 4s = 40 \) → \( s = 10 \)

3. **결론**:
   - 정사각형의 각 변의 길이는 \( 10m \)이며, 면적은 \( 100m^2 \)입니다.

### 경로 3: 산술-기하 평균을 이용한 접근

1. **문제 설정**:
   - 산술 평균과 기하 평균의 관계를 이용합니다.
   - \( \frac{x + y}{2} \geq \sqrt{xy} \)

2. **둘레 조건 적용**:
   - \( x + y = 2

In [ ]:
# Tree-of-Thought 예제 2: 화학 합성 경로
tot_problem_2 = """
다음 유기합성 문제를 Tree-of-Thought 방식으로 해결해주세요:

문제: 벤젠(C₆H₆)으로부터 아스피린(아세틸살리실산)을 합성하는 경로를 설계하세요.

다음과 같은 다양한 합성 전략을 고려해주세요:

전략 1: 페놀을 거치는 경로
- 벤젠 → 페놀 → 살리실산 → 아스피린

전략 2: 벤조산을 거치는 경로
- 벤젠 → 톨루엔 → 벤조산 → 살리실산 → 아스피린

전략 3: 직접 치환 반응 경로
- 벤젠에서 직접 다치환 반응

각 전략의 장단점을 평가하고 최적의 합성 경로를 추천해주세요.
반응 조건과 수율도 고려해주세요.
"""

messages = [
    {"role": "user", "content": tot_problem_2}
]

response = get_response(messages)
print_result("Tree-of-Thought", "화학 합성 경로", response)

## 7. [TODO]프롬프트 기법 비교 실험
지금까지 연습한 기법들을 바탕으로 동일한 task에 대해 다양한 prompt engineering 기법들을 직접 실습해 봅시다.

In [ ]:
# 비교 실험: 복잡한 물리 문제
comparison_problem = """
두 개의 블록(질량 m₁ = 2kg, m₂ = 3kg)이 마찰이 없는 수평면에서 용수철(k = 100 N/m)로 연결되어 있습니다.
처음에 용수철이 0.1m 압축된 상태에서 블록들을 놓았을 때, 용수철이 자연 길이가 되는 순간 각 블록의 속력을 구하세요.
"""

# your code


<details>
<summary>코드 예시 보기</summary>

```python
techniques = {
    "Zero-shot": comparison_problem,
    "Chain-of-Thought": comparison_problem + "\n\n단계별로 풀어주세요:",
    "Few-shot": f"""
유사한 문제 예시:
문제: 질량 1kg인 물체가 k=50N/m인 용수철에 0.2m 압축된 상태에서 놓였다. 속력은?
풀이: 에너지 보존 → ½kx² = ½mv² → v = √(kx²/m) = √(50×0.04/1) = 1.41m/s

이제 다음 문제를 풀어주세요:
{comparison_problem}
"""
}

print("같은 문제를 다른 프롬프트 기법으로 풀어보겠습니다...\n")

for technique, prompt in techniques.items():
    print(f"\n{'='*20} {technique} {'='*20}")
    
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    response = get_response(messages)
    print(response)
    print(f"{'='*50}")
    
    time.sleep(1)

print("고생하셨습니다.")
```
</details>

In [ ]:
# 비교 실험: 복잡한 수리 문제
comparison_problem = """
A회사에서는 제품을 생산하는 데 3단계의 공정을 거친다.

- 공정 1은 작업자 2명이 동시에 작업하며 6시간이 걸린다.

- 공정 2는 1명이 단독으로 작업하며 4시간이 걸린다.

- 공정 3은 작업자 3명이 함께 작업할 경우 2시간이 걸린다.

한 사람이 하루에 8시간까지 작업할 수 있을 때, 작업자 6명을 이용해 이 공정을 최대한 병렬로 운영했을 때, 하루에 몇 개의 제품을 만들 수 있는가?
"""

# your code

# 8. [TODO]프롬프트 엔지니어링으로 고난도 문제 풀기

여러 가지 방식을 활용하여 LLM으로 하여금 다음 문제를 풀어보게 합시다.
  
`!규칙!: OpenAI의 GPT-4o-mini 만을 사용합니다.`


In [ ]:
# 문제 1. 
# 정답 : 7 (a=3, f(1)=4)
problem1 = """
모든 실수 x에서 연속인 함수 f(x)에 대하여 $(x-1)f(x) = x^3 + 2x^2 -ax일 때, 상수 a와 f(1)에 대하여 a+ f(1)의 값은?$
"""

In [ ]:
# 문제 2. (24년 수능 화학1 12번) 
# 정답 2(5/6)
problem2 = """
다음은 2가지 산화 환원 반응에 대한 자료이다. 원소 X와 Y의 산화물에서 산소(O)의 산화수는 $-2$이다.
- **화학 반응식**
  (가)
  $$
  3XO_3^{3-} + BrO_3^- \\rightarrow 3XO_4^{3-} + Br^-
  $$
  (나)
  $$
  aX_2O_3 + 4YO_4^- + bH^+ \\rightarrow aX_2O_m + 4Y^{n+} + cH_2O (a \\sim c \\text{는 반응 계수})
  $$

- $\frac{생성물에서 X의 산화수}{반응물에서 X의 산화수}$는 (가)에서와 (나)에서가 같다.
- a는 (가)에서 각 원자의 산화수 중 가장 큰 값과 같다.

---

$$
\\frac{m \\times n}{b}
$$

은?
(단, X와 Y는 임의의 원소 기호이다.)

#### 보기:
1. $\\frac{2}{3}$
2. $\\frac{5}{6}$
3. $1$
4. $2$
5. $\\frac{5}{2}$
"""

In [ ]:
# 문제 3. 
# 정답 12
problem_3 = """다음 문장에는 알파벳 b가 몇 개 있나요?
betty and bobb brought back bright blue balloons from the big beach bazzar.
"""